In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import pickle as pkl
import warnings
import shutil
from collections import Counter
warnings.filterwarnings("ignore")

import torch
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from torchvision import datasets

from tqdm.notebook import tqdm

# Dataset download

In [5]:
!rm -r dataset
%mkdir dataset
!cp -v drive/MyDrive/TechSoc\ IHC\ 2021/track-1/dataset/dataset.zip dataset/dataset.zip
%cd dataset
#!gdown https://drive.google.com/uc?id=1if1LofYV6T_iKB2uphcDVnhrUGq_vgU1
#!gdown https://drive.google.com/uc?id=1WuwM5b-kG3-26EydEvwdMxeGeIBZ4Uix
!gdown https://drive.google.com/uc?id=1YV-B51V4_IsPMl9FVis4dhm_lUmsPtSX
!7z x dataset.zip
!7z x annotations.zip
!rm dataset.zip
!rm annotations.zip
%cd ..
%mkdir saved_weights
%mkdir submissions

'drive/MyDrive/TechSoc IHC 2021/track-1/dataset/dataset.zip' -> 'dataset/dataset.zip'
/content/dataset
Downloading...
From: https://drive.google.com/uc?id=1YV-B51V4_IsPMl9FVis4dhm_lUmsPtSX
To: /content/dataset/annotations.zip
100% 147k/147k [00:00<00:00, 43.5MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 15584540052 bytes (15 GiB)

Extracting archive: dataset.zip
 10% 4096 Open              --
Path = dataset.zip
Type = zip
Physical Size = 15584540052
64-bit = +

  0%      0% 48 - test/104.png                        0% 82 - test/1070.png                         0% 117 - test/1101.png                        

# Dataloader

## Train-Val loader

In [3]:
class IHC_train_dataset():
    def __init__(self, root: str, annFile: str, clsFile: str, transform = None):
        self.root = root
        self.annFile = annFile
        self.clsFile = clsFile
        self.transform = transform
        self.data = pkl.load(open(annFile, 'rb'))
        self.ids = list(self.data.keys())

    def __getitem__(self, index: int):
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image, class_id, class_val)
        """
        img_id = self.ids[index]
        target = self.data[img_id]
        image = Image.open(os.path.join(self.root, img_id)).convert('RGB')
        
        if self.transform is not None:
            image = self.transform(image)
        
        class_id = torch.tensor([target], dtype=torch.int64)
        
        return image, class_id


    def __len__(self):
        return len(self.ids)
    
    def __get_classes__(self):
        categories = pkl.load(open(self.clsFile, 'rb')) 
        return categories

In [4]:
class train_dataloader():
    def __init__(self, dataset, batch_size : int, shuffle : bool):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        if shuffle:
            self.index_sampler = torch.randperm(len(self.dataset))
        else:
            self.index_sampler = torch.tensor(list(range(len(self.dataset))))
        self.ids = iter(self.index_sampler)
        
    def _reset(self):
        self.index_sampler = torch.randperm(len(self.dataset))
        self.ids = iter(self.index_sampler)
    
    def _next_index(self):
        return next(self.ids)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        image_list = []
        class_id_list = []
        
        if self.ids is None:
            self._reset()
        
        for i in range(self.batch_size):
            id = self._next_index()
            image, class_id = self.dataset[id]
            
            image_list.append(torch.unsqueeze(image, 0))
            class_id_list.append(class_id)
        
        batch_images = torch.cat(image_list, 0)
        batch_class_ids = torch.cat(class_id_list, 0)

        return batch_images, batch_class_ids
    
    def __len__(self):
        return len(self.index_sampler)
    
    next = __next__

## Test loader

In [5]:
class IHC_test_dataset():
    def __init__(self, root: str, transform = None):
        self.root = root
        self.transform = transform
        self.ids = ['{0}.png'.format(i) for i in range(5000)]

    def __getitem__(self, index: int):
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image)
        """
        img_id = self.ids[index]
        image = Image.open(os.path.join(self.root, img_id)).convert('RGB')
        id = torch.tensor([int(img_id.split('.')[0])], dtype=torch.int64)
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, id


    def __len__(self):
        return len(self.ids)

In [6]:
class test_dataloader():
    def __init__(self, dataset, batch_size : int, shuffle : bool):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        if shuffle:
            self.index_sampler = torch.randperm(len(self.dataset))
        else:
            self.index_sampler = torch.tensor(list(range(len(self.dataset))))
        self.ids = iter(self.index_sampler)
        
    def _reset(self):
        self.index_sampler = torch.randperm(len(self.dataset))
        self.ids = iter(self.index_sampler)
    
    def _next_index(self):
        return next(self.ids)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        image_list = []
        image_id_list = []
        
        if self.ids is None:
            self._reset()
        
        for i in range(self.batch_size):
            id = self._next_index()
            image, image_id = self.dataset[id]
            
            image_list.append(torch.unsqueeze(image, 0))
            image_id_list.append(image_id)
        
        batch_images = torch.cat(image_list, 0)
        batch_image_ids = torch.cat(image_id_list, 0)

        return batch_images, batch_image_ids
    
    def __len__(self):
        return len(self.index_sampler)
    
    next = __next__

## Initialization

In [7]:
train_data = './dataset/train'
train_ann = './dataset/train_annotations.pkl'
class_file = './dataset/class_labels.pkl'

val_data ='./dataset/train'
val_ann ='./dataset/val_annotations.pkl'
class_file = './dataset/class_labels.pkl'

test_data ='./dataset/test'

train_transform = transforms.Compose([#transforms.Resize(256),
                                      #transforms.Resize(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                           std=[0.229, 0.224, 0.225]),
                                      ])
val_transform = transforms.Compose([#transforms.Resize(256),
                                    #transforms.Resize(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225]),
                                    ])
test_transform = transforms.Compose([#transforms.Resize(256),
                                     #transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                          std=[0.229, 0.224, 0.225]),
                                     ])

In [8]:
trainset = IHC_train_dataset(root=train_data, annFile=train_ann,
                             clsFile=class_file, transform=train_transform)
trainloader = train_dataloader(trainset, batch_size=50, shuffle=True)

valset = IHC_train_dataset(root=val_data, annFile=val_ann,
                           clsFile=class_file, transform=val_transform)
valloader = train_dataloader(valset, batch_size=50, shuffle=True)

testset = IHC_test_dataset(root=test_data, transform=val_transform)
testloader = test_dataloader(testset, batch_size=50, shuffle=False)

In [9]:
print('Number of training samples:', len(trainset))
print('Number of validation samples:', len(valset))
print('Number of test samples:', len(testset))
print('Categories :', trainset.__get_classes__())

Number of training samples: 28016
Number of validation samples: 6984
Number of test samples: 5000
Categories : {'real': 0, 'fake': 1}


# Model

In [10]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
from torch import Tensor
from torch.jit.annotations import List
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

__all__ = ['DenseNet', 'densenet121', 'densenet169', 'densenet201', 'densenet161']

model_urls = {
    'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
    'densenet169': 'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
    'densenet201': 'https://download.pytorch.org/models/densenet201-c1103571.pth',
    'densenet161': 'https://download.pytorch.org/models/densenet161-8d451a50.pth',
}


class _DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient

    def bn_function(self, inputs):
        # type: (List[Tensor]) -> Tensor
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
        return bottleneck_output

    # todo: rewrite when torchscript supports any
    def any_requires_grad(self, input):
        # type: (List[Tensor]) -> bool
        for tensor in input:
            if tensor.requires_grad:
                return True
        return False

    @torch.jit.unused  # noqa: T484
    def call_checkpoint_bottleneck(self, input):
        # type: (List[Tensor]) -> Tensor
        def closure(*inputs):
            return self.bn_function(inputs)

        return cp.checkpoint(closure, *input)

    @torch.jit._overload_method  # noqa: F811
    def forward(self, input):
        # type: (List[Tensor]) -> (Tensor)
        pass

    @torch.jit._overload_method  # noqa: F811
    def forward(self, input):
        # type: (Tensor) -> (Tensor)
        pass

    # torchscript does not yet support *args, so we overload method
    # allowing it to take either a List[Tensor] or single Tensor
    def forward(self, input):  # noqa: F811
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input

        if self.memory_efficient and self.any_requires_grad(prev_features):
            if torch.jit.is_scripting():
                raise Exception("Memory Efficient not supported in JIT")

            bottleneck_output = self.call_checkpoint_bottleneck(prev_features)
        else:
            bottleneck_output = self.bn_function(prev_features)

        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features


class _DenseBlock(nn.ModuleDict):
    _version = 2

    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, memory_efficient=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
    """

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000, memory_efficient=False):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        self.num_features = num_features
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        #print(features.shape)
        out = F.relu(features, inplace=True)
        #print(out.shape)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        #print(out.shape)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out


def _load_state_dict(model, model_url, progress):
    # '.'s are no longer allowed in module names, but previous _DenseLayer
    # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
    # They are also in the checkpoints in model_urls. This pattern is used
    # to find such keys.
    pattern = re.compile(
        r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')

    state_dict = load_state_dict_from_url(model_url, progress=progress)
    for key in list(state_dict.keys()):
        res = pattern.match(key)
        if res:
            new_key = res.group(1) + res.group(2)
            state_dict[new_key] = state_dict[key]
            del state_dict[key]
    model.load_state_dict(state_dict)


def _densenet(arch, growth_rate, block_config, num_init_features, pretrained, progress,
              **kwargs):
    model = DenseNet(growth_rate, block_config, num_init_features, **kwargs)
    if pretrained:
        _load_state_dict(model, model_urls[arch], progress)
    return model


def densenet121(pretrained=False, progress=True, **kwargs):
    r"""Densenet-121 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
    """
    return _densenet('densenet121', 32, (6, 12, 24, 16), 64, pretrained, progress,
                     **kwargs)


def densenet161(pretrained=False, progress=True, **kwargs):
    r"""Densenet-161 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
    """
    return _densenet('densenet161', 48, (6, 12, 36, 24), 96, pretrained, progress,
                     **kwargs)


def densenet169(pretrained=False, progress=True, **kwargs):
    r"""Densenet-169 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
    """
    return _densenet('densenet169', 32, (6, 12, 32, 32), 64, pretrained, progress,
                     **kwargs)


def densenet201(pretrained=False, progress=True, **kwargs):
    r"""Densenet-201 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
    """
    return _densenet('densenet201', 32, (6, 12, 48, 32), 64, pretrained, progress,
                     **kwargs)

# Train

In [11]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [12]:
def train_epoch(model, train_loader, optimizer, epoch):
    model.train()
    cls_loss = nn.NLLLoss()
    logsoftmax = nn.LogSoftmax(dim=1)
    total_cls_loss = 0.0
    correct = 0
    count = 0
    for i, data in tqdm(enumerate(train_loader)):
        images, targets = data
        images = images.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = cls_loss(logsoftmax(outputs), targets)
        
        loss.backward()
        optimizer.step()
        
        total_cls_loss += loss.item()
        correct += (predicted == targets).sum().item()
        count += targets.size(0)

        print('\r', 'Epoch : {0}, Iteration : {1} ==> Training loss : {2:.3f}, Accuracy : {3:.1f}%'.
                format(epoch+1, i+1, total_cls_loss/(i+1), 100*correct/count), end='')
        
    return model, total_cls_loss/(i+1)

# Validation

In [13]:
def val_epoch(model, val_loader, epoch):
    model.eval()
    cls_loss = nn.NLLLoss()
    logsoftmax = nn.LogSoftmax(dim=1)
    total_cls_loss = 0.0
    correct = 0
    count = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader)):
            images, targets = data
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            loss = cls_loss(logsoftmax(outputs), targets)
            
            total_cls_loss += loss.item()
            correct += (predicted == targets).sum().item()
            count += targets.size(0)

            print('\r', 'Epoch : {0}, Iteration : {1} ==> Validation loss : {2:.3f}, Accuracy : {3:.1f}%'.
                    format(epoch+1, i+1, total_cls_loss/(i+1), 100*correct/count), end='')
    
    return total_cls_loss/(i+1)

# Test

In [14]:
def test(model, test_loader, save_path):
    model.eval()
    test_ids = []
    test_p_real = []
    test_predictions = []
    softmax = torch.nn.Softmax(dim=1)
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader)):
            images, ids = data
            images = images.to(device)
            
            outputs = model(images)
            
            outputs = softmax(outputs)
            _, predicted = torch.max(outputs.data, 1)
            predicted = predicted.cpu().numpy()
            outputs = outputs[:, 0].squeeze().cpu().numpy()
            ids = ids.squeeze().cpu().numpy()
            
            test_ids.extend(list(ids))
            test_p_real.extend(list(outputs))
            test_predictions.extend(list(predicted))

    print('No. of images tested: {0}'.format(len(test_ids)))
    print('Distribution of classes:', Counter(test_predictions))
    print(max(test_p_real), min(test_p_real))

    df = pd.DataFrame(list(zip(test_ids, test_p_real)), columns=['id', 'p_real'])
    df.to_csv(save_path, index=False)

# Move saved files to drive

In [15]:
def move_data():
    dir = 'saved_weights'
    for file in os.listdir(dir):
        shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/track-1', dir, file))

    dir = 'submissions'
    for file in os.listdir(dir):
        shutil.copyfile(os.path.join(dir, file), os.path.join('drive/MyDrive/TechSoc IHC 2021/track-1', dir, file))

# DenseNet-121

In [21]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 20
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = densenet121(pretrained=True)
model.features.conv0 = nn.Conv2d(3, 64, kernel_size=11, stride=4,
                                 padding=2, bias=False)
model.classifier = nn.Linear(model.num_features, 2)
model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

Device : cuda


In [22]:
#valloader = train_dataloader(valset, batch_size=50, shuffle=True)
#vloss = val_epoch(model, valloader, epoch=-1)
    
for epoch in range(num_epochs):
    path = './saved_weights/pre_densenet-121.pt'
    
    trainloader = train_dataloader(trainset, batch_size=50, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=50, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch
        move_data()

 Epoch : 1, Iteration : 560 ==> Training loss : 0.307, Accuracy : 88.4%


 Epoch : 1, Iteration : 139 ==> Validation loss : 0.184, Accuracy : 93.3%
Saving....


 Epoch : 2, Iteration : 560 ==> Training loss : 0.147, Accuracy : 94.7%


 Epoch : 2, Iteration : 139 ==> Validation loss : 0.119, Accuracy : 95.8%
Saving....


 Epoch : 3, Iteration : 560 ==> Training loss : 0.112, Accuracy : 96.0%


...

 Epoch : 18, Iteration : 139 ==> Validation loss : 0.038, Accuracy : 98.6%
Saving....


 Epoch : 19, Iteration : 560 ==> Training loss : 0.027, Accuracy : 99.1%


 Epoch : 19, Iteration : 139 ==> Validation loss : 0.038, Accuracy : 98.7%
Saving....


 Epoch : 20, Iteration : 560 ==> Training loss : 0.027, Accuracy : 99.1%


 Epoch : 20, Iteration : 139 ==> Validation loss : 0.038, Accuracy : 98.6%


In [24]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=50, shuffle=True)
csv_path = './submissions/sub_22_4_21_pre_densenet-121.csv'

path = './saved_weights/pre_densenet-121.pt'
model = densenet121(pretrained=True)
model.features.conv0 = nn.Conv2d(3, 64, kernel_size=11, stride=4,
                                 padding=2, bias=False)
model.classifier = nn.Linear(model.num_features, 2)
model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)
move_data()


No. of images tested: 5000
Distribution of classes: Counter({1: 3392, 0: 1608})
0.9999931 3.3515116e-07


# DenseNet-161

In [28]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 20
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = densenet161(pretrained=True)
model.features.conv0 = nn.Conv2d(3, 96, kernel_size=11, stride=4,
                                 padding=2, bias=False)
model.classifier = nn.Linear(model.num_features, 2)
model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

Device : cuda


In [ ]:
#valloader = train_dataloader(valset, batch_size=50, shuffle=True)
#vloss = val_epoch(model, valloader, epoch=-1)
    
for epoch in range(num_epochs):
    path = './saved_weights/pre_densenet-161.pt'
    
    trainloader = train_dataloader(trainset, batch_size=25, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=25, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch
        move_data()

 Epoch : 1, Iteration : 215 ==> Training loss : 0.456, Accuracy : 82.4%

In [ ]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=25, shuffle=True)
csv_path = './submissions/sub_22_4_21_pre_densenet-161.csv'

path = './saved_weights/pre_densenet-161.pt'
model = densenet161(pretrained=True)
model.features.conv0 = nn.Conv2d(3, 96, kernel_size=11, stride=4,
                                 padding=2, bias=False)
model.classifier = nn.Linear(model.num_features, 2)
model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)
move_data()

# DenseNet-169

In [ ]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 20
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = densenet169(pretrained=True)
model.features.conv0 = nn.Conv2d(3, 64, kernel_size=11, stride=4,
                                 padding=2, bias=False)
model.classifier = nn.Linear(model.num_features, 2)
model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

In [ ]:
#valloader = train_dataloader(valset, batch_size=50, shuffle=True)
#vloss = val_epoch(model, valloader, epoch=-1)
    
for epoch in range(num_epochs):
    path = './saved_weights/pre_densenet-169.pt'
    
    trainloader = train_dataloader(trainset, batch_size=25, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=25, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch
        move_data()

In [ ]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=25, shuffle=True)
csv_path = './submissions/sub_22_4_21_pre_densenet-169.csv'

path = './saved_weights/pre_densenet-169.pt'
model = densenet169(pretrained=True)
model.features.conv0 = nn.Conv2d(3, 64, kernel_size=11, stride=4,
                                 padding=2, bias=False)
model.classifier = nn.Linear(model.num_features, 2)
model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)
move_data()

# DenseNet-201

In [ ]:
device = torch.device('cuda')
print("Device :", device)

num_epochs = 20
lr = 1e-4
best_vloss = 10
best_epoch = 0

model = densenet201(pretrained=True)
model.features.conv0 = nn.Conv2d(3, 64, kernel_size=11, stride=4,
                                 padding=2, bias=False)
model.classifier = nn.Linear(model.num_features, 2)
model = model.to(device)

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
schdl = torch.optim.lr_scheduler.CosineAnnealingLR(optim, num_epochs, eta_min=5e-6)

In [ ]:
#valloader = train_dataloader(valset, batch_size=50, shuffle=True)
#vloss = val_epoch(model, valloader, epoch=-1)
    
for epoch in range(num_epochs):
    path = './saved_weights/pre_densenet-201.pt'
    
    trainloader = train_dataloader(trainset, batch_size=25, shuffle=True)
    model, tloss = train_epoch(model, trainloader, optim, epoch)
    
    valloader = train_dataloader(valset, batch_size=25, shuffle=True)
    vloss = val_epoch(model, valloader, epoch)
    
    schdl.step()
    
    if vloss<best_vloss:
        print('Saving....')
        model_dict = {'epoch':epoch+1,
                      'model_state_dict':model.state_dict(),
                      'loss':vloss}
        torch.save(model_dict, path)
        best_vloss = vloss
        best_epoch = epoch
        move_data()

In [ ]:
testset = IHC_test_dataset(root=test_data, transform=test_transform)
testloader = test_dataloader(testset, batch_size=25, shuffle=True)
csv_path = './submissions/sub_22_4_21_pre_densenet-201.csv'

path = './saved_weights/pre_densenet-201.pt'
model = densenet201(pretrained=True)
model.features.conv0 = nn.Conv2d(3, 64, kernel_size=11, stride=4,
                                 padding=2, bias=False)
model.classifier = nn.Linear(model.num_features, 2)
model = model.to(device)
model_dict = torch.load(path)
model.load_state_dict(model_dict['model_state_dict'])

test(model, testloader, csv_path)
move_data()